In [11]:
from datetime import datetime

STRIKE_PRICE = 15.68
FMV_AT_EXERCISE = 19.95
CAPITAL_GAIN_TAX_RATE = 0.2
INCOME_TAX_RATE = 0.37
STATE_TAX_RATE = 0.1

GRANT_DATE = datetime.strptime('Mar 21 2021', '%b %d %Y')
MOVE_DATE = datetime.strptime('Mar 30 2022', '%b %d %Y')


In [12]:
import pandas as pd

class Event:
  def __init__(self, quantity, price, date, exercised=False, option_type="iso"):
    self.type = option_type
    self.quantity = quantity
    self.price = price
    self.date = datetime.strptime(date, '%b %d %Y')
    self.exercised = exercised

  def json(self):
    return {"gain": self.gain(), "tax": self.tax(), "cash": self.gain() - self.tax(), **self.__dict__}

  def gain(self):
    return int((self.price - (STRIKE_PRICE if not self.exercised else 0)) * self.quantity)

  def tax(self):
    # ISOs, 1) all capital gain 2) no state tax at sale
    if self.type == "iso":
      return int((self.price - STRIKE_PRICE) * self.quantity * CAPITAL_GAIN_TAX_RATE)

    # exercised NSOs, 1) all capital gain 2) no state tax at sale
    if self.exercised:
      return int((self.price - FMV_AT_EXERCISE) * self.quantity * CAPITAL_GAIN_TAX_RATE)
    
    ca_ratio = ((MOVE_DATE - GRANT_DATE).days + 14) * 1.0 / (self.date - GRANT_DATE).days
    return int((self.price - STRIKE_PRICE) * self.quantity * (ca_ratio * STATE_TAX_RATE + INCOME_TAX_RATE))


In [13]:
events = [
  Event(20500, 80, "Sep 30 2022", True, "nso"),
  Event(6377, 100, "Apr 01 2023", False, "iso"),
  Event(25000, 120, "Mar 15 2024", False, "nso"),
  Event(25000, 150, "Mar 15 2025", False, "nso"),
]

df = pd.DataFrame([e.json() for e in events])
df[["gain", "tax", "cash"]].sum()

gain    8143708
tax     2743547
cash    5400161
dtype: int64

In [14]:
df

,gain,tax,cash,type,quantity,price,date,exercised
0,1640000,246205,1393795,nso,20500,80,2022-09-30,True
1,537708,107541,430167,iso,6377,100,2023-04-01,False
2,2608000,1057795,1550205,nso,25000,120,2024-03-15,False
3,3358000,1332006,2025994,nso,25000,150,2025-03-15,False


In [15]:
events = [
  Event(20500, 70, "Sep 30 2022", True, "nso"),
  Event(6377, 70, "Apr 01 2023", False, "iso"),
  Event(25000, 80, "Mar 15 2024", False, "nso"),
  Event(25000, 90, "Mar 15 2025", False, "nso"),
]

df = pd.DataFrame([e.json() for e in events])
df
df[["gain", "tax", "cash"]].sum()

gain    5247396
tax     1663688
cash    3583708
dtype: int64

In [16]:
df

,gain,tax,cash,type,quantity,price,date,exercised
0,1435000,205205,1229795,nso,20500,70,2022-09-30,True
1,346398,69279,277119,iso,6377,70,2023-04-01,False
2,1607999,652198,955801,nso,25000,80,2024-03-15,False
3,1857999,737006,1120993,nso,25000,90,2025-03-15,False
